In [1]:
# %load main.py
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests


# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))


def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input = 'image_input:0'
    vgg_keep_prob = 'keep_prob:0'
    vgg_layer3_out = 'layer3_out:0'
    vgg_layer4_out = 'layer4_out:0'
    vgg_layer7_out = 'layer7_out:0'

    # Load the graph
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    graph = tf.get_default_graph()

    w1 = graph.get_tensor_by_name(vgg_input)
    keep = graph.get_tensor_by_name(vgg_keep_prob)
    w3 = graph.get_tensor_by_name(vgg_layer3_out)
    w4 = graph.get_tensor_by_name(vgg_layer4_out)
    w7 = graph.get_tensor_by_name(vgg_layer7_out)

    return w1, keep, w3, w4, w7


tests.test_load_vgg(load_vgg, tf)


/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


TensorFlow Version: 1.6.0
Default GPU Device: /device:GPU:0
Tests Passed


In [2]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    conv_1x1 = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, padding='same', 
                                kernel_initializer= tf.random_normal_initializer(stddev=0.01),
                                kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    output1 = tf.layers.conv2d_transpose(conv_1x1, num_classes, 4, 2, padding = 'same', 
                                        kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3))
    # Add a skip layer from a l4
    l4 = tf.layers.conv2d(vgg_layer4_out, num_classes, 1, padding='same', 
                          kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3))
    output2 = tf.add(output1, l4)

    # Upsample output to have the same size as l3
    output3 = tf.layers.conv2d_transpose(output2, num_classes, 4, 2, padding = 'same', 
                                        kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3))

    l3 = tf.layers.conv2d(vgg_layer3_out, num_classes, 1, padding='same', 
                          kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3))
    output4 = tf.add(output3, l3)

    # Upsample output to original image size

    output = tf.layers.conv2d_transpose(output4, num_classes, 16, 8, padding = 'same', 
                                        kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3))

    return output
print("Layers Test:")
tests.test_layers(layers)

Layers Test:
Tests Passed


In [3]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function

    logits = tf.reshape(nn_last_layer,(-1, num_classes))
    labels = tf.reshape(correct_label, (-1, num_classes))
    # define loss function
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= logits, labels= labels))
    # define training opertaion
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy_loss)
    return logits, train_op, cross_entropy_loss
print("Optimize Test:")
tests.test_optimize(optimize)

Optimize Test:
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Tests Passed


In [4]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    # TODO: Implement function
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(epochs):
        for image, label in get_batches_fn(batch_size):
            # Training process
            _, loss = sess.run([train_op, cross_entropy_loss], 
                                feed_dict = {input_image: image, correct_label: label, keep_prob: 0.5, 
                                             learning_rate: 1e-4 })
             # Print data on the learning process
            print("Epoch: {}".format(epoch + 1), "/ {}".format(epochs), " Loss: {:.3f}".format(loss))
tests.test_train_nn(train_nn)

INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'


In [5]:
def run():
    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)
    epochs = 48 # 6 12 24 
    batch_size = 5

    # Download pretrained vgg model
#     helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

        # TODO: Build NN using load_vgg, layers, and optimize function
        input_image, keep_prob, layer3_out, layer4_out, layer7_out = load_vgg(sess, vgg_path)
        layer_output = layers(layer3_out, layer4_out, layer7_out, num_classes)
        
        # TF placeholders for labels and learning rate
        correct_label = tf.placeholder(dtype = tf.int32, shape = (None, None, None, num_classes))
        learning_rate = tf.placeholder(dtype = tf.float32)
        
        # Creating loss and optimizer operations.
        logits, train_op, cross_entropy_loss = optimize(layer_output, correct_label, learning_rate, num_classes)

        # TODO: Train NN using the train_nn function

        # Create a model saver
#         saver = tf.train.Saver() 
        train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image, correct_label,
                 keep_prob, learning_rate)
         

        # TODO: Save inference data using helper.save_inference_samples
        #  helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)

        # OPTIONAL: Apply the trained model to a video


if __name__ == '__main__':
    run()